In [ ]:
import requests
from requests.exceptions import ConnectionError
from pyquery import PyQuery as pq
import pymongo


In [ ]:
client = pymongo.MongoClient('localhost')
db = client['yiou']

In [ ]:
def get_html(url):
    try:
        response= requests.get(url,allow_redirects=False)
        if response.status_code == 200:
            return response.text
    except ConnectionError:
        return get_html(url)

In [ ]:
def get_url(i):
    page=580
    url =  'https://www.iyiou.com/qiche/'+str(i)+'.html'
    html = get_html(url)
    return html

In [ ]:
def parse_url(html):
    doc = pq(html)
    items = doc('.industryPostList.borderE6 .newestArticleList li .text.fl a').items()
    for item in items:
        yield item.attr('href')

In [ ]:
def get_detail(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        return None
    except ConnectionError:
        return None

In [ ]:
def parse_detail(html):
    doc = pq(html)
    title = doc('#post_title').text()
    date = doc('#post_date').text()
    brief = doc('#post_brief').text()
    content = doc('#post_description').text()
    return{
        'title':title,
        'date':date,
        'brief':brief,
        'content':content
    }

In [ ]:
def save_to_mongo(data):
    if db['articles'].update({'title':data['title']},{'$set':data},True):
        print('Save to Mongo',data['title'])
    else:
        print('Save to Mongo Failed',data['title'])

In [ ]:
def main():
    for i in range(59,570):
        html = get_url(i)
        if html:
            article_urls = parse_url(html)
            for article_url in article_urls:
                article_html = get_detail(article_url)
                if article_html:
                    article_data = parse_detail(article_html)
                    print(article_data)
                    save_to_mongo(article_data)


In [ ]:
if __name__ == '__main__':
    main()